In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
import random
import pingouin as pg

from sklearn.model_selection import train_test_split
from matplotlib.ticker import FuncFormatter
import scipy.stats as stats
from statsmodels.formula.api import ols
import statsmodels.api as sm


# 한글 폰트 실행
from matplotlib import rcParams
from matplotlib import font_manager,rc
rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='NanumGothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)


In [2]:
import pandas as pd
DATA_PATH = './dataset/CARD/'
DATA_PATH_2 = './dataset/CUSTOMER/'
DATA_PATH_3 = './dataset/CC/'
DATA_PATH_4 = './dataset/CC/이전'
DATA_PATH_5 = './dataset/Tableau/'
#cc_21_40_raw = pd.read_csv(DATA_PATH_3 + 'cc_21_40(raw).csv', encoding="utf-8-sig")
#cc_21_50_raw = pd.read_csv(DATA_PATH_3 + 'cc_21_50(raw).csv', encoding="utf-8-sig")
#cc_21_60_raw = pd.read_csv(DATA_PATH_3 + 'cc_21_60(raw).csv', encoding="utf-8-sig")
#cc_22_40_raw = pd.read_csv(DATA_PATH_3 + 'cc_22_40(raw).csv', encoding="utf-8-sig")
#cc_22_50_raw = pd.read_csv(DATA_PATH_3 + 'cc_22_50(raw).csv', encoding="utf-8-sig")
#cc_22_60_raw = pd.read_csv(DATA_PATH_3 + 'cc_22_60(raw).csv', encoding="utf-8-sig")
#cc_23_40_raw = pd.read_csv(DATA_PATH_3 + 'cc_23_40(raw).csv', encoding="utf-8-sig")
#cc_23_50_raw = pd.read_csv(DATA_PATH_3 + 'cc_23_50(raw).csv', encoding="utf-8-sig")
#cc_23_60_raw = pd.read_csv(DATA_PATH_3 + 'cc_23_60(raw).csv', encoding="utf-8-sig")
#cc_22_40 = pd.read_csv(DATA_PATH_3 + 'cc_22_40.csv', encoding="utf-8-sig")
#cc_22_50 = pd.read_csv(DATA_PATH_3 + 'cc_22_50.csv', encoding="utf-8-sig")
#cc_22_60 = pd.read_csv(DATA_PATH_3 + 'cc_22_60.csv', encoding="utf-8-sig")

#cc_21_40_n = pd.read_csv(DATA_PATH_3 + 'cc_21_40_n.csv', encoding="utf-8-sig")
#cc_21_50_n = pd.read_csv(DATA_PATH_3 + 'cc_21_50_n.csv', encoding="utf-8-sig")
#cc_21_60_n = pd.read_csv(DATA_PATH_3 + 'cc_21_60_n.csv', encoding="utf-8-sig")

# 이전
#cc_22_40_n = pd.read_csv(DATA_PATH_4 + 'cc_22_40_n.csv', encoding="utf-8-sig")
#cc_22_50_n = pd.read_csv(DATA_PATH_4 + 'cc_22_50_n.csv', encoding="utf-8-sig")
#cc_22_60_n = pd.read_csv(DATA_PATH_4 + 'cc_22_60_n.csv', encoding="utf-8-sig")

#cc_22_n = pd.read_csv(DATA_PATH_4 + 'cc_22_n.csv', encoding="utf-8-sig")


#cc_22_40 = pd.read_csv(DATA_PATH_3 + 'cc_22_40.csv', encoding="utf-8-sig")
#cc_22_50 = pd.read_csv(DATA_PATH_3 + 'cc_22_50.csv', encoding="utf-8-sig")
#cc_22_60 = pd.read_csv(DATA_PATH_3 + 'cc_22_60.csv', encoding="utf-8-sig")

#cc_22_40_r = pd.read_csv(DATA_PATH_3 + 'cc_22_40_r.csv', encoding="utf-8-sig")
#cc_22_50_r = pd.read_csv(DATA_PATH_3 + 'cc_22_50_r.csv', encoding="utf-8-sig")
#cc_22_60_r = pd.read_csv(DATA_PATH_3 + 'cc_22_60_r.csv', encoding="utf-8-sig")

In [3]:
cc_23_40_r = pd.read_csv(DATA_PATH_3 + 'cc_23_40_r.csv', encoding="utf-8-sig")
cc_23_50_r = pd.read_csv(DATA_PATH_3 + 'cc_23_50_r.csv', encoding="utf-8-sig")
cc_23_60_r = pd.read_csv(DATA_PATH_3 + 'cc_23_60_r.csv', encoding="utf-8-sig")

# 층화추출

## 대분류

In [4]:
def strat(df, feature1, feature2, target):
    df_groupby = df.groupby([feature1, feature2])[target].sum().reset_index()
    df_drop = df_groupby.drop(columns=[feature1])
    counts = df_drop[feature2].value_counts()
    total_count = len(df_drop)
    sample_sizes = (counts / total_count * 10000).astype(int)  # 예시로 10000개 추출 기준 설정 (원하는 표본 크기 조정 가능)
    sample_df = df.groupby(feature2, group_keys=False).apply(
    lambda x: x.sample(n=sample_sizes[x.name])
    )
    aver_sample = sample_df.groupby(feature2)[target].mean()
    pd.options.display.float_format = '{:,.0f}'.format
    print(f"{feature2}별 연간 {target} 평균:")
    return aver_sample  # 반환값 추가

In [5]:
   """
    특정 그룹별로 표본을 추출하고, 그룹별 타겟 변수의 평균을 계산하는 함수.

    Parameters
    ----------
    df : pandas.DataFrame
        분석에 사용할 데이터프레임.
    feature1 : str
        첫 번째 그룹화에 사용할 열 이름.
    feature2 : str
        두 번째 그룹화에 사용할 열 이름. 주로 최종 표본 추출의 기준 그룹.
    target : str
        타겟 변수 이름. 평균을 계산할 열.

    Returns
    -------
    pandas.Series
        그룹별(target) 평균값을 포함하는 시리즈.

    Description
    -----------
    1. 데이터프레임을 `feature1`과 `feature2`로 그룹화하고, `target` 값의 합계를 계산합니다.
    2. `feature1` 열을 삭제하여 `feature2` 기준의 데이터프레임을 생성합니다.
    3. `feature2`별 값의 빈도를 계산하고, 전체 데이터 개수를 기준으로 표본 크기를 정합니다.
        - 표본 크기는 `counts / total_count * 10000`을 통해 계산되며, 여기서 10000은 표본 추출 기준값입니다.
        - 원하는 표본 크기에 따라 이 값을 조정할 수 있습니다.
    4. 각 `feature2` 그룹에서 설정한 크기만큼 표본을 무작위로 추출합니다.
    5. 추출된 표본으로부터 `feature2`별 타겟 변수(`target`)의 평균값을 계산합니다.
    6. 결과는 `feature2`별 타겟 변수의 연간 평균값을 포함하는 시리즈로 반환됩니다.
    
    Notes
    -----
    - `pd.options.display.float_format`는 출력 값을 포맷팅하여 보기 좋게 만듭니다.
    - `group_keys=False`를 사용하여 그룹화 결과에 불필요한 인덱스가 추가되지 않도록 합니다.
    - 표본 크기는 `10000`으로 설정되어 있으나, 필요에 따라 조정 가능합니다.

    Example
    -------
    >>> strat(df, '지역', '연령대', '소비금액')
    연령대별 연간 소비금액 평균:
    연령대
    20대    1,234,567
    30대    2,345,678
    40대    1,876,543
    Name: 소비금액, dtype: float64
    ```

SyntaxError: incomplete input (637975495.py, line 1)

## 가맹점업종명